In [1]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

2024-02-01 01:35:09.539322: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-01 01:35:09.580525: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-01 01:35:10.235269: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
import cv2  # OpenCVライブラリ
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import os
import sys
import pathlib
from pathlib import Path

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
import swallowing_recognition

In [4]:
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent

In [5]:
from swallowing_recognition import wavelet
from swallowing_recognition import audio
from swallowing_recognition import dataset
from swallowing_recognition import dence_net

In [6]:
directory_path = parent_dir / 'dataset'
   
train_voice_folder = directory_path / 'washino' / 'voice'
train_cough_folder = directory_path / 'washino' / 'cough'
train_swallowing_folder = directory_path / 'washino' / 'swallowing'   
train_movement_folder = directory_path / 'washino' / 'movement'   
train_hitting_folder = directory_path / 'washino' / 'hitting'   

test_voice_folder = directory_path / 'shibata' / 'voice'
test_cough_folder = directory_path / 'shibata' / 'cough'
test_swallowing_folder = directory_path / 'shibata' / 'swallowing'    

# 引数は(データ数，画像サイズ縦，画像サイズ横，色RGB，クラス数)
# train_data = dataset.DataSet(500, 224, 224, 3, 5)
# test_data = dataset.DataSet(70, 224, 224, 3, 5)

train_data = dataset.DataSet(500, 224, 224, 3, 2)
test_data = dataset.DataSet(28, 224, 224, 3, 2)

# train_data.folder_to_dataset(train_voice_folder, np.array([0, 0, 0, 0, 1]), 0)
# train_data.folder_to_dataset(train_cough_folder, np.array([0, 0, 0, 1, 0]), 1)
# train_data.folder_to_dataset(train_swallowing_folder, np.array([0, 0, 1, 0, 0]), 2)
# train_data.folder_to_dataset(train_movement_folder, np.array([0, 1, 0, 0, 0]), 3)
# train_data.folder_to_dataset(train_hitting_folder, np.array([1, 0, 0, 0, 0]), 4)

train_data.folder_to_dataset(train_swallowing_folder, np.array(0), 0)
train_data.folder_to_dataset(train_cough_folder, np.array(1), 100)
train_data.folder_to_dataset(train_voice_folder, np.array(1), 200)
train_data.folder_to_dataset(train_movement_folder, np.array(1), 300)
train_data.folder_to_dataset(train_hitting_folder, np.array(1), 400)

# test_data.folder_to_dataset(test_voice_folder, np.array([0, 0, 0, 0, 1]), 0)
# test_data.folder_to_dataset(test_cough_folder, np.array([0, 0, 0, 1, 0]), 1)
# test_data.folder_to_dataset(test_swallowing_folder, np.array([0, 0, 1, 0, 0]), 2)
# test_data.folder_to_dataset(test_cough_folder, np.array([0, 0, 0, 1, 0]), 3)
# test_data.folder_to_dataset(test_swallowing_folder, np.array([0, 0, 1, 0, 0]), 4)

test_data.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data.folder_to_dataset(test_cough_folder, np.array(1), 14)

In [7]:
# model = dence_net.DanceNet(5)

model = dence_net.DanceNet(2)

# 引数は(トレーニングデータ，ラベル，epochs，batch_size)
model.training(train_data.data, train_data.labels, 31, 30)
model.evaluate(test_data.data, test_data.labels)
model.save('20240131_binary_ep31_bs30.keras')

2024-02-01 01:37:38.950021: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22094 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:17:00.0, compute capability: 8.6


Epoch 1/31


2024-02-01 01:38:00.586734: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2024-02-01 01:38:02.348064: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-01 01:38:02.352302: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f7c26373770 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-01 01:38:02.352331: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-02-01 01:38:02.355986: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-01 01:38:02.476302: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

15/15 [==============================] - 58s 209ms/step - loss: 0.3292 - accuracy: 0.8756 - val_loss: 1.1050 - val_accuracy: 0.9400
Epoch 2/31
15/15 [==============================] - 1s 89ms/step - loss: 0.1647 - accuracy: 0.9422 - val_loss: 10.9966 - val_accuracy: 0.8400
Epoch 3/31
15/15 [==============================] - 1s 89ms/step - loss: 0.2037 - accuracy: 0.9222 - val_loss: 0.0848 - val_accuracy: 0.9800
Epoch 4/31
15/15 [==============================] - 1s 93ms/step - loss: 0.1172 - accuracy: 0.9556 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/31
15/15 [==============================] - 1s 92ms/step - loss: 0.0782 - accuracy: 0.9733 - val_loss: 15.6899 - val_accuracy: 0.1600
Epoch 6/31
15/15 [==============================] - 1s 89ms/step - loss: 0.0340 - accuracy: 0.9911 - val_loss: 2.8573 - val_accuracy: 0.5800
Epoch 7/31
15/15 [==============================] - 1s 89ms/step - loss: 0.0199 - accuracy: 0.9911 - val_loss: 0.8367 - val_accuracy: 0.7600
Epoch 8/31
15/15

In [8]:
model.evaluate_print()

[[6.5146283e-05]
 [7.3560608e-05]
 [1.6498740e-03]
 [2.3142109e-06]
 [9.9996734e-01]
 [1.8284446e-02]
 [2.0712318e-03]
 [4.4655362e-03]
 [9.5257890e-01]
 [7.1820714e-02]
 [1.1397384e-04]
 [9.3130268e-07]
 [1.1394884e-04]
 [9.7800177e-01]
 [9.9999177e-01]
 [9.9999750e-01]
 [9.9998641e-01]
 [9.9998903e-01]
 [9.9998987e-01]
 [9.9770039e-01]
 [9.9998677e-01]
 [9.9999785e-01]
 [9.9999797e-01]
 [9.9996901e-01]
 [9.9998724e-01]
 [9.9999762e-01]
 [9.9996376e-01]
 [9.9994946e-01]]
[0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True False False False False False False False False False False
  False False False False]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True False False False False False False False False False False
  False False False False]
 [ True  True  True  True  T